# Tidy Data 

Key Components of tidy data: 
- data is tabular, i.e. made up of rows and columns 
- there is one value per cell 
- each variable is a column 
- each observation is a row 

In [10]:
import pandas as pd
import env
import pydataset 

DB_URL = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/tidy_data'

One Variable in Multiple Columns

In [2]:
treatments = pd.read_sql('SELECT * FROM treatments', DB_URL)
treatments

,patient,treatmenta,treatmentb,treatmentc
0,John Smith,NaN,2,0
1,Jane Doe,16.0,11,3
2,Mary Johnson,3.0,1,4


In [3]:
#rename columns 
treatments.columns = ['name', 'a', 'b', 'c']
treatments

,name,a,b,c
0,John Smith,NaN,2,0
1,Jane Doe,16.0,11,3
2,Mary Johnson,3.0,1,4


In [4]:
#melt 
#combine the last three columns in the DataFrame 
treatments.melt(id_vars=['name'], var_name='treatment', value_name= 'response')

,name,treatment,response
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0
6,John Smith,c,0.0
7,Jane Doe,c,3.0
8,Mary Johnson,c,4.0


One Column with Multiple Variables

In [5]:
students = pd.read_sql('SELECT * FROM students', DB_URL)
students.head(9)

,date,var,val
0,2019-02-04,n_late_from_break,4.028120
1,2019-02-04,coffee_consumption,5255.409740
2,2019-02-04,classroom_temp,67.000000
3,2019-02-05,n_late_from_break,2.101998
4,2019-02-05,coffee_consumption,8603.704719
5,2019-02-05,classroom_temp,73.000000
6,2019-02-06,n_late_from_break,4.941244
7,2019-02-06,coffee_consumption,1801.498050
8,2019-02-06,classroom_temp,81.000000


This DataFrame is an example of a colum that was melted incorrectly. 'n_late_from_break', 'classroom_temp', and 'coffee_consumption' should be their own columns- this can be achieved in one of two ways:

### 1

In [6]:
students.pivot(index='date', columns='var')

val                                     
var        classroom_temp coffee_consumption n_late_from_break
date                                                          
2019-02-04           67.0        5255.409740          4.028120
2019-02-05           73.0        8603.704719          2.101998
2019-02-06           81.0        1801.498050          4.941244
2019-02-07           62.0        9282.959741          1.419342
2019-02-08           72.0        7558.270659          1.808919
2019-02-11           60.0        5731.008713          4.831584
2019-02-12           52.0        9547.673484          3.550830
2019-02-13           56.0        7114.931847          2.550362
2019-02-14           72.0        1977.295513          3.507548
2019-02-15           75.0        2254.674679          2.114379
2019-02-18           55.0        4819.981878          2.249609
2019-02-19           82.0        8628.442171          4.646607
2019-02-20           58.0        6905.019187          1.309552
2019-02-21           79.0        1414.705565          2.132542
2019-02-22           68.0        8894.846285          1.745758
2019-02-25           84.0        5982.280540          2.344274
2019-02-26           54.0        9121.391375          4.620919
2019-02-27           88.0        4412.125903          1.860681
2019-02-28           81.0        1693.233409          1.342123

### 2

In [7]:
students.set_index(['date', 'var']).unstack(level=1)

val                                     
var        classroom_temp coffee_consumption n_late_from_break
date                                                          
2019-02-04           67.0        5255.409740          4.028120
2019-02-05           73.0        8603.704719          2.101998
2019-02-06           81.0        1801.498050          4.941244
2019-02-07           62.0        9282.959741          1.419342
2019-02-08           72.0        7558.270659          1.808919
2019-02-11           60.0        5731.008713          4.831584
2019-02-12           52.0        9547.673484          3.550830
2019-02-13           56.0        7114.931847          2.550362
2019-02-14           72.0        1977.295513          3.507548
2019-02-15           75.0        2254.674679          2.114379
2019-02-18           55.0        4819.981878          2.249609
2019-02-19           82.0        8628.442171          4.646607
2019-02-20           58.0        6905.019187          1.309552
2019-02-21           79.0        1414.705565          2.132542
2019-02-22           68.0        8894.846285          1.745758
2019-02-25           84.0        5982.280540          2.344274
2019-02-26           54.0        9121.391375          4.620919
2019-02-27           88.0        4412.125903          1.860681
2019-02-28           81.0        1693.233409          1.342123

In [9]:
#Remove the multiindex to make working with the resulting DataFrame easier
tidy_students = students.set_index(['date', 'var']).unstack(level=1)
tidy_students.columns = tidy_students.columns.droplevel()
tidy_students.columns.name = ''

tidy_students

,classroom_temp,coffee_consumption,n_late_from_break
date,,,
2019-02-04,67.0,5255.409740,4.028120
2019-02-05,73.0,8603.704719,2.101998
2019-02-06,81.0,1801.498050,4.941244
2019-02-07,62.0,9282.959741,1.419342
2019-02-08,72.0,7558.270659,1.808919
2019-02-11,60.0,5731.008713,4.831584
2019-02-12,52.0,9547.673484,3.550830
2019-02-13,56.0,7114.931847,2.550362
2019-02-14,72.0,1977.295513,3.507548
